In [50]:
import pandas as pd
import pyodbc
import re

In [80]:
sql_conn = pyodbc.connect('DRIVER={SQL Server};'
                            'SERVER=L_AAGDATIX;'
                            'DATABASE=DatixCRM;'
                            'Trusted_Connection=yes') 
query = "set transaction isolation level read uncommitted select inc_reportedby,inc_notes from DatixCRM.dbo.incidents_main where inc_type='PAT' and inc_category='MEDIC' and inc_organisation='QA'"
df = pd.read_sql(query, sql_conn)
df.head()

,inc_reportedby,inc_notes
0,None,on drug rounds found Amlodipine not given for ...
1,None,HAEMOPHILIA PATIENT\r\n\r\nPATIENT NOT ADMINIS...
2,Other,"Resus controlled order booked went missing, ha..."
3,Other (Nurse / Midwife ),MIDAZOLAM SIGNED OUT OF CD BOOK FOR PATIENT . ...
4,Nurse,I was giving the oral medication(CD-oxycodone)...


In [81]:
roles = list(df['inc_reportedby'].value_counts().index)

In [82]:
if '' in roles: roles.remove('')
print(roles)

['Nurse', 'Pharmacist', 'Pharmacy technician', 'Administration / clerical / reception staff', 'Midwife', 'Physician', 'Doctor (unknown grade)', 'Radiographer', 'Health Care Support Worker (HCSW)', 'Pharmacy staff in training ', 'Anaesthetist', 'Foundation Year Doctor (First Year) (FY1)', 'Dietician', 'Other (Allied health professional)', 'Other (Nurse / Midwife )', 'Manager', 'Nurse / Midwife  in training ', 'Foundation Year Doctor (Second Year) (FY2)', 'Physiotherapist', 'Surgeon', 'Other (Support staff (clinical and administration)', 'Dispensing / pharmacy assistant', 'Allied health professional in training / supervision', 'Other (Pharmacy staff)', 'Other (Medical staff)', 'Other', 'General practitioner (GP) - registrar', 'Nursery nurse', 'Scientific and technical', 'Occupational therapist', 'Student (Non Medical)', 'Research fellow', 'Physiotherapy Support Worker/Technician', 'Speech & language therapist', 'Facilties Management (FM) Staff (All Types)', 'Support staff in training ']


In [109]:
split_slash = []
for role in roles:
    for r in role.split('/'):
        split_slash.append(r.strip())
split_bracket = []
for role in split_dash:
    for r in role.split(' ('):
        split_bracket.append(r.strip(' )'))
role_search = []
for role in split_bracket:
    for r in role.split(' - '):
        role_search.append(r.strip(')'))
role_search = [role.replace('  ',' ') for role in role_search]
role_search = [role.lower().replace(' staff','') for role in role_search]
role_search = [role.strip(')') for role in role_search]
if 'unknown grade' in role_search: role_search.remove('unknown grade')
if 'all types' in role_search: role_search.remove('all types')
role_search = list(filter(lambda r: r!='other', role_search))

seen = set()
seen_add = seen.add
role_search = [x for x in role_search if not (x in seen or seen_add(x))]

role_search = [role.replace(' & ',' and ') for role in role_search]
role_search.append('dr')
role_search.append('1st year')
role_search.append('2nd year')
for i,role in enumerate(role_search):
    if re.sub(r"(\s)and(\s)",r"\1&\2",role)!=role: role_search.append(re.sub(r"(\s)and(\s)",r"\1&\2",role))
    #if re.sub(r"(\s)year",r"\1yr",role)!=role: role_search.append(re.sub(r"(\s)year",r"\1yr",role))
#for role in role_search:
#    if re.sub(r"administration",r"admin",role)!=role: role_search.append(re.sub(r"administration",r"admin",role))
role_search.remove('pharmacy')
print(role_search)

['nurse', 'pharmacist', 'pharmacy technician', 'administration', 'clerical', 'reception', 'midwife', 'physician', 'doctor', 'radiographer', 'health care support worker', 'hcsw', 'pharmacy in training', 'anaesthetist', 'foundation year doctor', 'first year', 'fy1', 'dietician', 'allied health professional', 'manager', 'midwife in training', 'second year', 'fy2', 'physiotherapist', 'surgeon', 'support', 'clinical and administration', 'dispensing', 'pharmacy assistant', 'allied health professional in training', 'supervision', 'medical', 'general practitioner', 'gp', 'registrar', 'nursery nurse', 'scientific and technical', 'occupational therapist', 'student', 'non medical', 'research fellow', 'physiotherapy support worker', 'technician', 'speech and language therapist', 'facilties management', 'fm', 'support in training', 'dr', '1st year', '2nd year', 'clinical & administration', 'scientific & technical', 'speech & language therapist']


In [103]:
role_num_dict = {}
for role in role_search:
    df_role = df[df['inc_notes'].str.lower().str.contains(' '+role+' ')]
    role_num_dict[role] = len(df_role)
    print(df_role['inc_notes'].values)

["I had spoken to the oncall Gastro ?SHO aprox 10:30 Saturday 2nd April via ward C5 extention number with regards to an oulired patients medication being reviewed, I informed them that the patients bloods were last done on the 31/03/16. The SHO stated that it was the responsibility of our ward SHO to cover outlires at the weekend. I said that this is not a practice I or our ward SHO is aware of. The Gastrso SHO said that she was on ward round but would come and review the patient at some point in the day.\r\nI contacted ward C5 on Sunday 3rd April at 1520 to see if the patient was going to be reviewed as she had not been the previous day, I spoke to a staff nurse on the ward who stated the SHO was busy with a poorly patient, I asked a message to be passed on that the SHO had been contacted yesterday morning with regards to the patients medication being reviewed and they have yet to be seen, I was assured the message would be passed on. \r\nAt 1720 I contacted ward C5 for the oncall's b

 'Baby coming from B7 for IV antibiotics Flucloxacillin and Gentamicin, antibiotics due at 01:00. Neonatal doctor phone maternity to ask them to bring baby round to B9 for antibiotics to be given. Baby brought by mum at 01:55. Medications checked and prepared when checking for identification baby had no name bands on. Phone B7 to explain cannot give medication without name bands and asked midwife to come label baby. This delayed given gentamicin so not given within 1 hour of prescribed time.  ']
['During an external site Occupational Health Clinic an employee in an external company not healthcare, was given 4th Dose of Hepatitis B and 3rd dose of Hepatitis A vaccine in error. The companies risk assessments identified employees required occupational vaccines due to the nature of their work.\r\nThe contributory factors included recording vaccines in paper files and on the computer. At  previous visit in December 2015 the Nurse had recorded the vaccines on the computer but not in the indi

 'Patient received on D5 after a Total Right Knee replacement on the 7/11/2019. On arrival from Recovery, handed over by Recovery Nurse to me that the patient has a clothing disorder (documented) and so no anticoagulants were prescribed. Also handed over that for that reason the patient should be reviewed by Haematology in 2 days. Anaesthetist came to the ward before 20h00 to prescribe IV antibiotics and informed me that Nursing staff should chase the doctors regarding the anticoagulant every day. On the 8/11/19 Orthopaedic ward team chased regarding it and prescribed Clexane 40mg to start on the 10/11/19. Haematology not chased. Today (10/11/19) discussed with doctor on call regarding the Haematology review: noticed on the system (not on the paper medical notes) that the Haematology review had been done previously to surgery and that a plan had been formulated already: Transemic Acid 1gram been given previously to surgery and to be given on the 5 days after surgery, and clexane to be 

[]
[]
[]
["Known asthmatic had surgery for acute appendicitis. Found on ward round on 2/2/19 to be wheezy and salbutamol nebuliser written up to be given 4 times a day. Instructions given to nurse to give nebuliser and to get physiotherapist to see patient. \r\nDiscovered on ward round of 4/2/19 that patient's chest much worse and that the prescribed nebuliser had only been given once and no physiotherapist had seen patient."
 'Incident received through external mailbox to risk management department from St James Hospital, via solent nhs:\r\n\r\n"Patient was admitted to PHT on 06/02/19 for infective exacerbation of COPD. On discharge on 03/03/19 to Jubilee House for\r\nCHC assessment PHT discharge summary incorrectly stated Patient has Long Term Oxygen Therapy (LTOT) at home. This was\r\nremoved in September 2017. The discharge summary acknowledges use of oxygen at point of admission to ED but not following\r\nnor advice for on going oxygen requirements at Jubilee House. The discharge 

 'Tried to order 10 x IV metronidazole on a ward sheet yesterday for C6 stock, but was contacted by the pharmacy lead to say she will not be dispensing it as it was ordered for stock by the top up team. No IV metronidazole arrived on the ward yesterday, C6 nurses had to borrow doses from multiple different wards in order to avoid missing the doses. \r\n\r\nChecked the ward distribution list for C6 ward stock today, no IV metronidazole was ordered by the top up team. ']
['Reported by pharmacy 11/1/16, changed location for E2 to review 1/2/17.(KD)\r\n\r\n\r\n2x4 complete boxes of Buprenorphine patches 10mcg & 5mcg found in E2 medication returns box by pharmacy assistant who was returning unused medication from this ward. .'
 "reported by chronic pain team 24/10/18 changed to pharmacy location 14/11/18 as slight delay to dispensing\r\n\r\nThe patient had attended for an intrathecal pump refill. A staff nurse went to collect the morphine syringe from Pharmacy and provided the pharmacy assi

['Patients eye drops were prescribed and given in the right eye. This was queried as all other eye drops were for the left. The ophthalmology registrar confirmed that the drops were for the left.'
 'Transferred to B8 to continue induction of labour for Type 2 diabetes at 36+6 weeks for large for dates. Owing to the precipitate labour in first pregnancy it was decided to administer intravenous antibiotics prior to artificially rupturing membranes as GBS positive. Almost as soon as Benzyl penicillin started a bout of sneezing, face and abdomen flushed red and reported feeling unwell. IV antibiotics stopped. Woman vomited copious amount and then after walking to toilet felt drowsy. Emergency buzzer pressed and 2nd on call registrar attended and the 2nd on call anaesthetist fast bleeped. Pathological CTG but primary concern was to stabilise maternal health'
 'Patient was admitted to F7 on the 1/7/2016 and a stat dose of IV Tazocin 4.5g was given. It was documented in the patients medical n

[]
['The patient was prescribed AZARGA eyedrops by his GP which were entered on to the Drug Chart.  The AZARGA brinzolamide and timolol eyedrops were ordered from Pharmacy.  GANFORT bimatoprost and timolol eyedrops were dispensed and labelled as AZARGA  in Pharmacy, and the error was not detected by the Checking Technician and the wrong eyedrops were sent to the ward.  The patient has been administered the GANFORT eyedrops since 22-4-2016'
 'Patient brought prescription from ED for co-amoxiclav. When giving out to patient technician checked if allergic and patient said was allergic to all penicillins and had a red wrist band on. \r\nScript also had no date on the prescription ( CD requirement for tramadol) '
 "Above patient's medicine transferred from C5 to E2 with another patient. Put into patient's POD locker on E2 and was found by Medicines Management Technician whilst DHx taking and discharge planning. "
 'Adult Cardiac Arrest boxes were being checked prior to sealing, Technician s

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
['Patient (& wife) called by name from waiting room and brought to clinic room to commence induction BCG. Procedure explained and patient labelled consent form shown and talked through. Drug chart signed for no known allergies. Patient was asked if they had any questions, which they did not. BCG delivered with patient consent.\r\nAfter the patient had left the department what transpired to be the correct patient made themselves known to a member of the admin team. I immediately contacted my line manager and cancer lead for Urology.'
 'Patient admitted to B6 on 27th Oct. Obtained drug history from patient who stated she was on Insulatard insulin.  This was not prescribed on drug chart/patient did not have pink diabetic drug chart. Wrote note for dr on front of drug chart, informed midwife and left message for diabetic midwives to review this patient and others for diabetic meds to be prescribed/reviewed.\r\nVisited ward again on Mon 31st Oct. Patient now ha

[]


In [104]:
print(role_num_dict)

{'nurse': 1146, 'pharmacist': 600, 'pharmacy technician': 22, 'administration': 230, 'clerical': 1, 'reception': 31, 'midwife': 43, 'physician': 5, 'doctor': 457, 'radiographer': 14, 'health care support worker': 1, 'hcsw': 29, 'pharmacy in training': 0, 'anaesthetist': 33, 'foundation year doctor': 0, 'first year': 0, 'fy1': 18, 'dietician': 5, 'allied health professional': 0, 'manager': 36, 'midwife in training': 0, 'second year': 0, 'fy2': 0, 'physiotherapist': 2, 'surgeon': 13, 'support': 44, 'clinical and administration': 0, 'dispensing': 107, 'pharmacy assistant': 3, 'allied health professional in training': 0, 'supervision': 4, 'medical': 293, 'general practitioner': 1, 'gp': 204, 'registrar': 67, 'nursery nurse': 2, 'scientific and technical': 0, 'occupational therapist': 0, 'student': 19, 'non medical': 0, 'research fellow': 0, 'physiotherapy support worker': 0, 'technician': 63, 'speech and language therapist': 1, 'facilties management': 0, 'fm': 19, 'support in training': 0,